In [93]:
import pandas as pd
import os

In [94]:
allgsdf = pd.read_csv("../../resources/20180622processedGoldStandardTopics.tsv.gz", delimiter="\t")
gsdf = allgsdf.drop(["title", "abstract", "major_mesh", "minor_mesh"], axis=1)

# Read all results

In [106]:
def calculateRelFoundCounts(dir):
    # Read all results in the given result directory
    resultfiles = sorted(os.listdir(dir))
    # Read the result files as DataFrames into a map
    resultdfmap = []
    for f in resultfiles:
        resultdfmap.append(pd.read_csv(dir+f, delimiter="\t", names=["topic", "Q0", "docid", "rank", "score", "run"]))
    # Create a DataFrame multiindexed with the file name (because those are the keys of the DF maps)
    resultmultidxdf = pd.concat(resultdfmap)
    resultmultidxdf.set_index(["run"], inplace=True)
    
    # Create a duplication of the relevant GS document to match the results
    gsreldocs = gsdf.query("relevance_score > 0")[["trec_topic_number", "trec_doc_id"]]
    l = []
    for experiment in set(resultmultidxdf.index):
        idx = pd.Index([experiment]*len(gsreldocs), name="run")
        gscopy = gsreldocs.copy()
        gscopy.index = idx
        l.append(gscopy)
    gsdfs = pd.concat(l)
    
    # Merge the duplicated GS with the results
    # With a `left` join, thus eliminating all irrelevant documents.
    relmerge = pd.merge(gsdfs, resultmultidxdf, how="left", left_on=["run", "trec_topic_number", "trec_doc_id"], right_on=["run", "topic", "docid"])
    relmerge.set_index("trec_topic_number", append="True", inplace=True)
    
    # Count the number of found documents per run and topic    
    countsruntopic = relmerge.groupby(["run", "trec_topic_number"]).count()
    countsruntopic = countsruntopic.drop(["Q0", "docid", "rank", "score"],axis=1)
    countsruntopic.columns = ["relgs", "relfound"]
    
    # return the left-merged data and the counts
    return relmerge,countsruntopic

In [96]:
def prepareStats(path):
    df = pd.read_csv(path).query("Topic != 'all'")
    df["Topic"] = df["Topic"].astype(int)
    df.sort_values(by="Topic", inplace=True)
    df = df.set_index("Topic")
    return df

In [123]:
measures = ["infNDCG"]

def getMeanStatsPerRun(statspath):
    statfiles = sorted(list(filter(lambda f: f.endswith(".csv") and not "gspm" in f, os.listdir(statspath))))
    runstatsmap = {}
    for stat in statfiles:
        df = prepareStats(statspath+stat)
        run = stat.replace("OFFICIAL_", "").replace(".csv", "")
        runstatsmap[run] = df
    allstats = pd.concat(runstatsmap)
    allstats.index.names = ["run", "Topic"]
    allstats = allstats[measures]
    meanstats = allstats.mean(level="run")
    return meanstats

In [124]:
meanstatsnowr = getMeanStatsPerRun("../../stats_nowr/")
meanstatswr = getMeanStatsPerRun("../../stats_wr/")

In [141]:
gsleftmergedwr,countswr = calculateRelFoundCounts("../../results_wr/")
countswr    = countswr.sum(level="run").sort_values("relfound")
wrmerge     = pd.merge(countswr, meanstatswr, on="run")
meanrankswr = gsleftmergedwr["rank"].dropna().mean(level="run")
stdrankswr  = gsleftmergedwr["rank"].dropna().std(level="run")
wrmerge["meanrank"] = meanrankswr
wrmerge["stdrank"] = stdrankswr
wrmerge.sort_values("relfound")

,relgs,relfound,infNDCG,meanrank,stdrank
run,,,,,
genesonly,3875,2771,0.382883,829.558643,1549.346101
baselinewr,3875,2898,0.444370,587.314010,1000.123354
diseases,3875,2979,0.408020,654.488083,1121.057310
genesnodesc,3875,3174,0.454193,643.141462,1019.549901
genes,3875,3183,0.444593,713.724788,1202.682173
genesplus,3875,3184,0.453837,682.587626,1100.730490
baseline,3875,3228,0.393510,760.732962,1216.394281
genedis,3875,3252,0.387483,823.507688,1368.686248


In [145]:
print(wrmerge.to_latex(encoding='utf-8'))

\begin{tabular}{lrrrrr}
\toprule
{} &  relgs &  relfound &   infNDCG &    meanrank &      stdrank \\
run         &        &           &           &             &              \\
\midrule
genesonly   &   3875 &      2771 &  0.382883 &  829.558643 &  1549.346101 \\
baselinewr  &   3875 &      2898 &  0.444370 &  587.314010 &  1000.123354 \\
diseases    &   3875 &      2979 &  0.408020 &  654.488083 &  1121.057310 \\
genesnodesc &   3875 &      3174 &  0.454193 &  643.141462 &  1019.549901 \\
genes       &   3875 &      3183 &  0.444593 &  713.724788 &  1202.682173 \\
genesplus   &   3875 &      3184 &  0.453837 &  682.587626 &  1100.730490 \\
baseline    &   3875 &      3228 &  0.393510 &  760.732962 &  1216.394281 \\
genedis     &   3875 &      3252 &  0.387483 &  823.507688 &  1368.686248 \\
\bottomrule
\end{tabular}



In [138]:
gsleftmergednowr,countsnowr = calculateRelFoundCounts("../../results_nowr/")
countsnowr = countsnowr.sum(level="run").sort_values("relfound")
nowrmerge = pd.merge(countsnowr, meanstatsnowr, on="run")
meanranksnowr = gsleftmergednowr["rank"].dropna().mean(level="run")
stdranksnowr = gsleftmergednowr["rank"].dropna().std(level="run")
nowrmerge["meanrank"] = meanranksnowr
nowrmerge["stdrank"] = stdranksnowr
nowrmerge.sort_values("relfound")

,relgs,relfound,infNDCG,meanrank,stdrank
run,,,,,
baselinewr,3875,2898,0.444370,587.314010,1000.123354
genesonly,3875,2989,0.326227,949.805621,1517.841900
baseline,3875,3228,0.393510,760.732962,1216.394281
diseases,3875,3324,0.344807,923.909446,1408.774797
genes,3875,3472,0.371300,935.288594,1419.388215
genesnodesc,3875,3481,0.383080,877.321172,1308.963602
genesplus,3875,3482,0.377230,922.338024,1378.319279
genedis,3875,3541,0.315567,1153.769557,1656.313819


In [146]:
print(nowrmerge.to_latex(encoding='utf-8'))

\begin{tabular}{lrrrrr}
\toprule
{} &  relgs &  relfound &   infNDCG &     meanrank &      stdrank \\
run         &        &           &           &              &              \\
\midrule
baselinewr  &   3875 &      2898 &  0.444370 &   587.314010 &  1000.123354 \\
genesonly   &   3875 &      2989 &  0.326227 &   949.805621 &  1517.841900 \\
baseline    &   3875 &      3228 &  0.393510 &   760.732962 &  1216.394281 \\
diseases    &   3875 &      3324 &  0.344807 &   923.909446 &  1408.774797 \\
genes       &   3875 &      3472 &  0.371300 &   935.288594 &  1419.388215 \\
genesnodesc &   3875 &      3481 &  0.383080 &   877.321172 &  1308.963602 \\
genesplus   &   3875 &      3482 &  0.377230 &   922.338024 &  1378.319279 \\
genedis     &   3875 &      3541 &  0.315567 &  1153.769557 &  1656.313819 \\
\bottomrule
\end{tabular}



In [139]:
pd.merge(wrmerge, nowrmerge, on="run", suffixes=["_wr", "_nowr"])

,relgs_wr,relfound_wr,infNDCG_wr,meanrank_wr,stdrank_wr,relgs_nowr,relfound_nowr,infNDCG_nowr,meanrank_nowr,stdrank_nowr
run,,,,,,,,,,
genesonly,3875,2771,0.382883,829.558643,1549.346101,3875,2989,0.326227,949.805621,1517.841900
baselinewr,3875,2898,0.444370,587.314010,1000.123354,3875,2898,0.444370,587.314010,1000.123354
diseases,3875,2979,0.408020,654.488083,1121.057310,3875,3324,0.344807,923.909446,1408.774797
genesnodesc,3875,3174,0.454193,643.141462,1019.549901,3875,3481,0.383080,877.321172,1308.963602
genes,3875,3183,0.444593,713.724788,1202.682173,3875,3472,0.371300,935.288594,1419.388215
genesplus,3875,3184,0.453837,682.587626,1100.730490,3875,3482,0.377230,922.338024,1378.319279
baseline,3875,3228,0.393510,760.732962,1216.394281,3875,3228,0.393510,760.732962,1216.394281
genedis,3875,3252,0.387483,823.507688,1368.686248,3875,3541,0.315567,1153.769557,1656.313819
